## Multi-Information Source BO with Augmented Gaussian Processes
- Contributors: andreaponti5
- Last updated: Jan 29, 2024
- BoTorch version: 0.9.5(dev)

In this tutorial, we show how to perform Multiple Information Source Bayesian Optimization in BoTorch based on the Augmented Gaussian Process (AGP) and the Augmented UCB (AUCB) acquisition function proposed in [1].
The key idea of the AGP is to fit a GP model for each information source and *augment* the observations on the high fidelity source with those from *cheaper* sources which can be considered as *reliable*. The GP model fitted on this *augmented* set of observations is the AGP.
The AUCB is a modification of the standard UCB -- computed on the AGP -- suitably proposed to also deal with the source-specific query cost.

We emprically show that the *AGP-based* Multiple Information Source Basyesian Optimization usually performs better than other multi-fidelity approaches [2].

[1] [Candelieri, A., & Archetti, F. (2021). Sparsifying to optimize over multiple information sources: an augmented Gaussian process based algorithm. Structural and Multidisciplinary Optimization, 64, 239-255.](https://link.springer.com/article/10.1007/s00158-021-02882-7)
[2] [The arxiv will be available soon.](https://arxiv.org/)


In [1]:
import importlib
required_packages = ['matplotlib', 'numpy', 'torch', 'transformers', 'datasets', 'botorch', 'gpytorch', 'scipy']

for package in required_packages:
    try:
        importlib.import_module(package)
    except ImportError:
        !pip install {package}


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.1/159.1 kB 766.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 1.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.9/303.9 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 1.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 1.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 1.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 2.3 MB/s eta 0:00:00 0:00:01
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.

: 

In [3]:
import os
import matplotlib.pyplot as plt

import torch
from gpytorch import ExactMarginalLogLikelihood

import botorch
from botorch import fit_gpytorch_mll
from botorch.acquisition import InverseCostWeightedUtility, qMultiFidelityMaxValueEntropy
from botorch_community.acquisition.augmented_multisource import AugmentedUpperConfidenceBound
from botorch.models import AffineFidelityCostModel, SingleTaskMultiFidelityGP
from botorch_community.models.gp_regression_multisource import SingleTaskAugmentedGP, get_random_x_for_agp
from botorch.models.transforms import Standardize
from botorch.optim import optimize_acqf, optimize_acqf_mixed
from botorch.test_functions.multi_fidelity import AugmentedBranin

In [4]:
tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}
SMOKE_TEST = os.environ.get("SMOKE_TEST", False)

In [5]:
N_ITER = 10 if SMOKE_TEST else 50
SEED = 3

### Problem setup
We consider the augmented Branin multi-fidelity synthetic test problem. It is important to clarify that *augmented* is not about the AGP: here, it has a different meaning. It means that the Branin test function has been modified by introducing an additional dimension representing the fidelity parameter.

The test function takes the form $f(x,s)$ where $x \in [-5, 10] \times [0, 15]$ and $s \in [0,1]$. The target fidelity is 1.0, which means that our goal is to solve $\max_x f(x,1.0)$ by making use of cheaper evaluations $f(x,s)$ for $s < 1.0$. In this example, we'll assume that the cost function takes the form $5.0 + s$, illustrating a situation where the fixed cost is $5.0$.

Since a multiple information source context is considered, three different sources are considered, with $s = 0.5, 0.75, 1.00$, respectively.

In [57]:
problem = AugmentedBranin(negate=True).to(**tkwargs)
fidelities = torch.tensor([0.5, 0.75, 1.0], **tkwargs)
n_sources = fidelities.shape[0]

# dim = self.dim
# max_idx =self.max_idx
n_sources = 3
dim = 10
max_idx = 11001
bounds = torch.tensor([[0] * (dim + 1), [max_idx] * (dim + 1)], dtype=torch.float64)
bounds[-1,-1] = (n_sources - 1)
print(f"\t Bound = {bounds};")

problem = AugmentedBranin(negate=True).to(**tkwargs)
target_fidelities = {n_sources - 1: 1.0}
cost_model = AffineFidelityCostModel(fidelity_weights=target_fidelities, fixed_cost=1.1)
cost_aware_utility = InverseCostWeightedUtility(cost_model=cost_model)

	 Bound = tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.1001e+04, 1.1001e+04, 1.1001e+04, 1.1001e+04, 1.1001e+04, 1.1001e+04,
         1.1001e+04, 1.1001e+04, 1.1001e+04, 1.1001e+04, 2.0000e+00]],
       dtype=torch.float64);


In [58]:
# problem = AugmentedBranin(negate=True).to(**tkwargs)
# fidelities = torch.tensor([0.5, 0.75, 1.0], **tkwargs)
# n_sources = fidelities.shape[0]
# print(n_sources)
# bounds = torch.tensor([[-5, 0, 0], [10, 15, n_sources - 1]], **tkwargs)
# target_fidelities = {n_sources - 1: 1.0}

# cost_model = AffineFidelityCostModel(fidelity_weights=target_fidelities, fixed_cost=5.0)
# cost_aware_utility = InverseCostWeightedUtility(cost_model=cost_model)

### Model initialization

We use a `SingleTaskAugmentedGP` to implement our AGP.

At each Bayesian Optimization iteration, the set of observations from the *ground-truth* (i.e., the highest fidelity and more expensive source) is temporarily *augmented* by including observations from the other cheap sources, only if they can be considered *reliable*. Specifically, an observation $(x,y)$ from a cheap source is considered reliable if it satisfies the following inequality:

$$\vert\mu(x)-y\vert \leq m \sigma(x)$$

where $\mu(x)$ and $\sigma(x)$ are, respectively, the posterior mean and standard deviation of the GP model fitted on the high fidelity observations only, and $m$ is a technical parameter making more *conservative* ($m→0$) or *inclusive* ($m→∞)$ the augmentation process. As reported in [1], a suitable value for this parameter is $m=1$.

After the set of observations is augmented, the AGP is fitted through `SingleTaskAugmentedGP`.


In [59]:
def generate_initial_data(n):
    train_x = get_random_x_for_agp(n, bounds, 1)
    xs = train_x[..., :-1]
    fids = fidelities[train_x[..., -1].int()].reshape(-1, 1)
    train_obj = problem(torch.cat((xs, fids), dim=1)).unsqueeze(-1)
    return train_x, train_obj


def initialize_model(train_x, train_obj, m):
    model = SingleTaskAugmentedGP(
        train_x, train_obj, m=m, outcome_transform=Standardize(m=1),
    )
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    return mll, model

#### Define a helper function that performs the essential BO step
This helper function optimizes the acquisition function and returns the candidate point along with the observed function values.

The UCB acquisition function has been modified to deal with both the *discrepancy* between information sources and the *source-specific query cost*.

Formally, the AUCB acquisition function, at a generic iteration $t$, is defined as:

$$\alpha_s(x,\hat y^+) = \frac{\left[\hat{\mu}(x) + \sqrt{\beta^{(t)}} \hat{\sigma}(x)\right] - \hat{y}^+}{c_s \cdot (1+\vert \hat{\mu}(x) - \mu_s(x) \vert)} $$

where $\hat{y}^+$ is the best (i.e., highest) value in the *augmented* set of observations, the numerator is -- therefore -- the optimistic improvement with respect to $\hat{y}^+$, $c_s$ is the query cost for the source $s$, and $\vert \hat{\mu}(x) - \mu_s(x) \vert$ is a discrepancy measure between the predictions provided by the AGP and the GP on the source $s$, respectively, given the input $x$ (i.e., 1 is added just to avoid division by zero).

For more information, please refer to [1],

In [60]:
def optimize_aucb(acqf):
    candidate, value = optimize_acqf(
        acq_function=acqf,
        bounds=bounds,
        q=1,
        num_restarts=5,
        raw_samples=128,
    )
    # observe new values
    new_x = candidate.detach()
    new_x[:, -1] = torch.round(new_x[:, -1], decimals=0)
    return new_x

### Perform a few steps of multi-fidelity BO
First, let's generate some initial random data and fit a surrogate model.

In [61]:
torch.manual_seed(SEED)
train_x, train_obj = generate_initial_data(n=5)

In [62]:
train_x, train_obj

(tensor([[7.5318e+02, 2.4700e+03, 7.6917e+03, 6.8851e+02, 4.1297e+03, 3.3466e+02,
          1.6212e+03, 3.7243e+03, 2.9720e+03, 2.9985e+03, 1.0000e+00],
         [7.5227e+03, 1.0500e+04, 4.7819e+03, 1.0585e+04, 8.1834e+03, 9.8811e+03,
          8.3993e+03, 7.5654e+03, 7.8570e+03, 8.9943e+03, 1.0000e+00],
         [1.0915e+04, 5.1708e+03, 8.3359e+03, 5.3681e+03, 1.0536e+04, 6.1054e+03,
          5.4461e+03, 1.0128e+04, 1.0755e+04, 7.2204e+03, 0.0000e+00],
         [2.7732e+03, 7.7972e+03, 1.2140e+03, 6.6498e+03, 1.8598e+03, 4.6482e+03,
          6.5511e+03, 5.6050e+02, 4.1736e+02, 2.6023e+03, 2.0000e+00],
         [5.1908e+03, 3.5431e+03, 3.6410e+03, 7.4364e+03, 9.7679e+02, 2.6772e+03,
          3.3168e+03, 9.4177e+03, 9.5916e+03, 1.0627e+04, 0.0000e+00]],
        dtype=torch.float64),
 tensor([[-1.9041e+17],
         [-7.3237e+20],
         [-9.8641e+21],
         [-8.7202e+17],
         [-9.6257e+19]], dtype=torch.float64))

We can now use the helper functions above to run a few iterations of BO.

In questo problema ci sono 3 fonti,  0 ha costo 5.5, 1 ha costo 7.75 e 2 costo 6.00
    {i: fid + 5.0 for i, fid in enumerate(fidelities)} -> {0: tensor(5.5000, dtype...h.float64), 1: tensor(5.7500, dtype...h.float64), 2: tensor(6., dtype=tor...h.float64)}
L'ultimo valore di ogni x rappresenta la fonte id Fid 
    new_x[:, -1][0] -> 1.00  |>>| ter 0;	 Fid = 1.00;	 Obj = -200.3252;

dunque train_x ha (in questo caso) le prime due dimensioni che rappresentano le informazioni del punto e la terza che rappresenta la source
    tensor([[-3.9730,  3.3679,  1.0000],
            [ 3.2453, 10.8819,  1.0000],
            [ 8.0466,  7.0303,  2.0000],
            [-1.0395, 14.4851,  0.0000],
            [ 1.8883,  4.8380,  1.0000],
            [ 5.0308, 14.9492,  2.0000],
            [ 8.0466,  7.0303,  2.0000],
            [ 8.0466,  7.0302,  2.0000]], dtype=torch.float64)

In [63]:
cumulative_cost = 0.0

with botorch.settings.validate_input_scaling(False):
    for it in range(N_ITER):
        mll, model = initialize_model(train_x, train_obj, m=1)
        fit_gpytorch_mll(mll)
        acqf = AugmentedUpperConfidenceBound(
            model,
            beta=3,
            maximize=True,
            best_f=train_obj[torch.where(train_x[:, -1] == 0)].min(),
            cost={i: fid + 5.0 for i, fid in enumerate(fidelities)},
        )
        new_x = optimize_aucb(acqf)
        if model.n_true_points < model.max_n_cheap_points:
            new_x[:, -1] = fidelities.shape[0] - 1
        train_x = torch.cat([train_x, new_x])

        new_x[:, -1] = fidelities[new_x[:, -1].int()]
        new_obj = problem(new_x).unsqueeze(-1)
        train_obj = torch.cat([train_obj, new_obj])

        print(
            f"Iter {it};"
            f"\t Fid = {new_x[0].tolist()[-1]:.2f};"
            f"\t Obj = {new_obj[0][0].tolist():.4f};"
        )

Iter 0;	 Fid = 1.00;	 Obj = -25848880479644758016.0000;
Iter 1;	 Fid = 1.00;	 Obj = -400313830635190336.0000;


/opt/conda/lib/python3.11/site-packages/botorch/optim/fit.py:102: OptimizationWarning: `scipy_minimize` terminated with status OptimizationStatus.FAILURE, displaying original message from `scipy.optimize.minimize`: ABNORMAL_TERMINATION_IN_LNSRCH
  warn(


Iter 2;	 Fid = 1.00;	 Obj = -702897826604901597184.0000;


/opt/conda/lib/python3.11/site-packages/botorch/optim/fit.py:102: OptimizationWarning: `scipy_minimize` terminated with status OptimizationStatus.FAILURE, displaying original message from `scipy.optimize.minimize`: ABNORMAL_TERMINATION_IN_LNSRCH
  warn(


Iter 3;	 Fid = 1.00;	 Obj = -5789263563085260800.0000;
Iter 4;	 Fid = 1.00;	 Obj = -194146102532375707648.0000;
Iter 5;	 Fid = 1.00;	 Obj = -1048828111299228729344.0000;
Iter 6;	 Fid = 1.00;	 Obj = -50779831892422492160.0000;


...

Iter 13;	 Fid = 1.00;	 Obj = -26785928349472698368.0000;


/opt/conda/lib/python3.11/site-packages/botorch/optim/fit.py:102: OptimizationWarning: `scipy_minimize` terminated with status OptimizationStatus.FAILURE, displaying original message from `scipy.optimize.minimize`: ABNORMAL_TERMINATION_IN_LNSRCH
  warn(
/opt/conda/lib/python3.11/site-packages/botorch/optim/fit.py:102: OptimizationWarning: `scipy_minimize` terminated with status OptimizationStatus.FAILURE, displaying original message from `scipy.optimize.minimize`: ABNORMAL_TERMINATION_IN_LNSRCH
  warn(


Iter 14;	 Fid = 1.00;	 Obj = -1497650117723517091840.0000;


/opt/conda/lib/python3.11/site-packages/botorch/optim/fit.py:102: OptimizationWarning: `scipy_minimize` terminated with status OptimizationStatus.FAILURE, displaying original message from `scipy.optimize.minimize`: ABNORMAL_TERMINATION_IN_LNSRCH
  warn(


KeyboardInterrupt: 